# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
B. Rajpoot  ->  B. Rajpoot  |  ['B. Rajpoot']
M. Cecil  ->  M. Cecil  |  ['M. Cecil']
M. Flock  ->  M. Flock  |  ['M. Flock']
P. Sudarshan  ->  P. Sudarshan  |  ['P. Sudarshan']
N. Martin  ->  N. Martin  |  ['N. Martin']
N. Martin  ->  N. Martin  |  ['N. Martin']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 63 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.10260


extracting tarball to tmp_2602.10260... done.


G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
B. Rajpoot  ->  B. Rajpoot  |  ['B. Rajpoot']


Issues with the citations
plugin pybtex.database.input.suffixes for suffix .tex not found
Retrieving document from  https://arxiv.org/e-print/2602.10674
extracting tarball to tmp_2602.10674...

...

 done.


Retrieving document from  https://arxiv.org/e-print/2602.10883
extracting tarball to tmp_2602.10883...

 done.


N. Martin  ->  N. Martin  |  ['N. Martin']


Issues with the citations
repeated bibliography entry: noauthor_notitle_nodate
Retrieving document from  https://arxiv.org/e-print/2602.10895
extracting tarball to tmp_2602.10895... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.10260-b31b1b.svg)](https://arxiv.org/abs/2602.10260) | **Cold and eccentric: a high-spectral resolution view of 51 Eri b with VLT/HiRISE**  |
|| A. Denis, et al. -- incl., <mark>G. Chauvin</mark>, <mark>M. Ravet</mark>, <mark>B. Rajpoot</mark> |
|*Appeared on*| *2026-02-12*|
|*Comments*| *9 pages, 8 figures, Accepted for publication in A&A*|
|**Abstract**|            Discovered almost 10 years ago, the giant planet 51 Eridani b is one of the least separated (0.2 arcsec) and faintest (J = 19.74 mag) directly imaged exoplanets known to date. Its atmospheric properties have been thoroughly investigated through low- and medium-resolution spectroscopic observations, enabling robust characterization of the planet's bulk parameters. However, the planet's intrinsically high contrast renders high-resolution spectroscopic observations difficult, despite their potential to yield key measurements essential for a more comprehensive characterization. This study seeks to constrain the planet's radial velocity, enabling a full 3D orbital solution when integrated with previous measurements. We have obtained 4 high-contrast high-resolution (R = 140000) spectroscopic datasets of the planet, collected over a two-year interval with the HiRISE visitor instrument at the VLT to derive the planet's radial velocity. Using self-consistent models of atmosphere, we were able to derive the radial velocity of the planet at each of the 4 epochs. These radial velocity measurements were then used in combination with all existing relative astrometry in order to constrain the orbit of the planet. Our radial velocity measurements allow us to break the degeneracy along the line of sight, making it now possible the unambiguous interpretation of the phase curve of the companion. We further constrain the orbital parameters, particularly the eccentricity, for which we derive e = 0.55 (-0.07, +0.03). The relatively high eccentricity indicates that the system has experienced dynamical interactions induced by an external perturber. We place constraints on the mass and semi-major axis of a hypothetical, unseen outer planet capable of producing the observed high eccentricities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.10674-b31b1b.svg)](https://arxiv.org/abs/2602.10674) | **The role of detailed gas and dust opacities in shaping the evolution of the inner disc edge subject to episodic accretion**  |
|| <mark>M. Cecil</mark>, et al. -- incl., <mark>M. Flock</mark>, <mark>P. Sudarshan</mark> |
|*Appeared on*| *2026-02-12*|
|*Comments*| *22 pages, 15 figures, accepted for publication in A&A*|
|**Abstract**|            We investigate the effects of different dust and gas opacity descriptions on the structure and evolution of the inner regions of protoplanetary discs. The influence on the episodic instability of the inner rim is hereby of central interest. 2D axisymmetric radiation hydrodynamic models are employed to simulate the evolution of the inner disc over several thousand years. Our simulations greatly expand on previous models by implementing detailed opacity descriptions in terms of their mean and frequency-dependent values, allowing us to also consider binned frequency-dependent irradiation. The adaptive opacity description significantly affects the structure of the inner disc rim, with gas opacities exerting the greatest influence. The resulting effects include shifts in the position of both the dust sublimation front and the dead zone inner edge, a significantly altered temperature in the dust-free region and the manifestation of an equilibrium temperature degeneracy as a sharp temperature transition. The episodic instability due to MRI activation in the dead zone still occurs, but at lower inner disc densities. While the gas opacities set the initial conditions for the instability, the evolution of the outburst itself is mainly governed by the dust opacities. The analysis of criteria for non-axisymmetric instabilities reveals possible breaking of the density peaks produced by the burst. However, due to the periodicity of the instability, the inner edge itself may remain stable throughout quiescent phases according to linear criteria. Although the thermal structure of the inner disc is crucially affected by different opacity descriptions, the mechanism of the periodic instability of the DZIE remains active and is only marginally influenced by gas opacities. The observational consequences of the severely altered temperatures may be significant and require further investigation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.10859-b31b1b.svg)](https://arxiv.org/abs/2602.10859) | **The metal-poor tail of the APOGEE survey I. Uncovering [Fe/H] < -2.5 stars from the inner Galaxy to the Magellanic Clouds**  |
|| M. Montelius, et al. -- incl., <mark>N. Martin</mark> |
|*Appeared on*| *2026-02-12*|
|*Comments*| **|
|**Abstract**|            In the search for metal-poor stars, large spectroscopic surveys are an invaluable tool. However, the spectra of metal-poor stars can be difficult to analyse because of the relative lack of available lines, which can also lead to misclassification. We aim to identify the stars observed by the APOGEE survey that are below the metallicity limit of APOGEE's analysis. For the highest confidence candidates, we classify the orbital properties of the stars to investigate whether their orbital distribution matches what we would expect for stars that are this metal poor and to select especially interesting targets for spectroscopic follow-up purposes. We examined the properties derived by APOGEE for metal-poor stars from the literature to find signatures of stars with a metallicity below the range of the grid used for spectral analysis. The calibrated APOGEE stellar parameters provide a clear signature of the most metal-poor stars in the survey, indicated by null values for their metallicities while having effective temperatures and surface gravities determined by the pipeline. From comparison with the literature, we find that, within a temperature range of 3700 - 6700 K and above a threshold of S/N > 30, the vast majority of APOGEE stars without calibrated metallicities are very metal poor. The radial velocities provided by APOGEE, Gaia DR3 positions and astrometry along with spectrophotometric distances derived in this work allowed for the computation of their orbits. In this work, we select 289 very metal-poor red giant stars (likely below = -2.5) from the APOGEE results. Our sample contains 16 very metal-poor member candidates of the Magellanic Clouds, 14 very metal-poor stars with orbits confined to the inner Galaxy, and 13 inner Galaxy halo interlopers.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.10883-b31b1b.svg)](https://arxiv.org/abs/2602.10883) | **The metal-poor tail of the APOGEE survey II. Spectral analysis of Mg and Si in very metal-poor APOGEE spectra**  |
|| M. Montelius, et al. -- incl., <mark>N. Martin</mark> |
|*Appeared on*| *2026-02-12*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            H-band spectra contain very limited spectral information for stars at the most metal-poor tail ( Fe/H < -2.5) because the available Fe lines in FGK stars in this wavelength range are weak. The first paper in this series successfully identified a sample of 327 very metal-poor stars (with [Fe/H] < -2) from the APOGEE database, 289 of which are on the red giant branch. The spectra of these stars were not properly analysed by the APOGEE main pipeline because they are very metal poor. In this work, we measure metallicities for these stars using the abundances of the elements Mg and Si. We demonstrate that the absorption lines of the elements Mg and Si are of good quality despite the challenging combination of (low) metallicity, wavelength regime, spectral resolution, and signal-to-noise ratios available for these spectra. A specialised pipeline was designed to measure the abundance of Mg and Si in APOGEE spectra and yielded a robust estimate of the overall metallicity. In order to provide reliable measurements, we tested three different sets of assumptions for Mg and Si enhancement. We present Mg and Si abundances as well as overall metallicities for 327 stars, all of which had previously gotten null values from the main APOGEE pipeline for either the calibrated M/H or [Fe/H] . The typical uncertainties for our measurements are 0.2 dex. We found five stars in our sample with unusual [Si/Mg] abundances higher than 0.5, and we connect this signature to globular cluster stars, and this might be related to specific supernova events. Our data suggest a concentration of high [Si/Mg] stars in the Sextans dwarf galaxy. Other dwarf galaxies are found to agree well with results in the literature. Our derived metallicities range between -3.1 $\leq$ [M/H] $\leq$ -2.25, thereby pushing the metal-poor tail of APOGEE results down by 0.6 dex.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.10895-b31b1b.svg)](https://arxiv.org/abs/2602.10895) | **GECAM discovery of the second FRB-associated Magnetar X-ray Burst from SGR J1935+2154**  |
|| C.-W. Wang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-02-12*|
|*Comments*| *Accepted by MNRAS*|
|**Abstract**|            Fast radio burst (FRB) is mysterious phenomenon with millisecond-duration radio pulses observed mostly from cosmological distance. The association between FRB 200428 and a magnetar X-ray burst (MXB) from SGR J1935+2154 has significantly advanced the understanding of FRB and magnetar bursts. However, it is uncertain whether this association between MXB and FRB (i.e. MXB/FRB 200428) is genuine or just coincidental only based on this single event. Here we report the discovery of a bright ($\rm\sim7.6\times10^{-7}\,erg \cdot cm^{-2}$ in 1-250 keV) magnetar X-ray burst detected by GECAM on October 14th, 2022 (dubbed as MXB 221014) from SGR J1935+2154, which is associated with a FRB detected by CHIME and GBT. We conducted a detailed temporal and spectral analysis of MXB 221014 with GECAM data and find that it is a bright and typical ($T_{90}\sim$250 ms) X-ray burst from this magnetar. Interestingly, we find two narrow X-ray pulses in the MXB, one of which temporally aligns with the main pulse of the FRB 221014 $\sim5.70$ ms latter than the peak time of FRB 221014), resembling the feature found in MXB/FRB 200428. Furthermore, we did comprehensive comparison between MXB/FRB 221014 and MXB/FRB 200428, and find that while the two events share several common features, they also exhibit distinct differences, highlighting the variety of the MXB-FRB association morphology. This finding not only confirms the association between MXB and FRB but also provides new insights into the mechanism of and the relationship between FRB and MXB.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.10260/./Figures/astrometry_RV.png', 'tmp_2602.10260/./Figures/corner_51Eri.png', 'tmp_2602.10260/./Figures/Fringes.png']
copying  tmp_2602.10260/./Figures/astrometry_RV.png to _build/html/
copying  tmp_2602.10260/./Figures/corner_51Eri.png to _build/html/
copying  tmp_2602.10260/./Figures/Fringes.png to _build/html/
exported in  _build/html/2602.10260.md
    + _build/html/tmp_2602.10260/./Figures/astrometry_RV.png
    + _build/html/tmp_2602.10260/./Figures/corner_51Eri.png
    + _build/html/tmp_2602.10260/./Figures/Fringes.png
found figures ['tmp_2602.10674/./figures/fig_burststructure_h2rmid_newopac.png', 'tmp_2602.10674/./figures/fig_old_new_diff.png', 'tmp_2602.10674/./figures/fig_rootfinder_combined_new.png']
copying  tmp_2602.10674/./figures/fig_burststructure_h2rmid_newopac.png to _build/html/
copying  tmp_2602.10674/./figures/fig_old_new_diff.png to _build/html/
copying  tmp_2602.10674/./figures/fig_rootfinder_combined_new.png to _build/html/
exported in

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rev}[1]{\textcolor{\textbf{purple}}{#1}}$
$\newcommand{\MSun}{\ensuremath{M_{\odot}}\xspace}$
$\newcommand{\MJup}{\ensuremath{M_{\mathrm{Jup}}}\xspace}$
$\newcommand{\RJup}{\ensuremath{R_{\mathrm{Jup}}}\xspace}$
$\newcommand{\Teff}{\ensuremath{T_{\mathrm{e\!f\!f}}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log g}\xspace}$
$\newcommand{\met}{\ensuremath{\mathrm{[Fe/H]}}\xspace}$
$\newcommand{\co}{\ensuremath{\mathrm{C/O}}\xspace}$
$\newcommand{\mic}{\ensuremath{\upmu\mathrm{m}}\xspace}$
$\newcommand{\as}{\hbox{^{\prime\prime}}\xspace}$
$\newcommand{\vsini}{\hbox{v \sin i}\xspace}$
$\newcommand{\loD}{\hbox{\lambda/D}\xspace}$
$\newcommand{◦e}{◦ee\xspace}$
$\newcommand{◦eC}{◦ee C\xspace}$
$\newcommand{\microrad}{\ensuremath{\upmu\mathrm{rad}}\xspace}$
$\newcommand{\kms}{\ensuremath{\mathrm{km} \mathrm{s}^{-1}}\xspace}$
$\newcommand{\crires}{CRIRES\xspace}$
$\newcommand{\hirise}{HiRISE\xspace}$
$\newcommand{\ms}{\ensuremath{\mathrm{m} \mathrm{s}^{-1}}\xspace}$
$\newcommand{\formosa}{\texttt{ForMoSA}\xspace}$
$\newcommand{\uves}{UVES\xspace}$
$\newcommand{\jwst}{\textit{James Webb} Space Telescope\xspace}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Eccentric and cool: a high-spectral resolution view of\\51 Eri b with VLT/HiRISE$\footnote{Based on observations made with ESO Telescopes at the La Silla Paranal Observatory under programmes ID 112.25FU, 114.2712, and 115.284P}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.10260-b31b1b.svg)](https://arxiv.org/abs/2602.10260)<mark>Appeared on: 2026-02-12</mark> -  _9 pages, 8 figures, Accepted for publication in A&A_

</div>
<div id="authors">

A. Denis, et al. -- incl., <mark>G. Chauvin</mark>, <mark>M. Ravet</mark>, <mark>B. Rajpoot</mark>

</div>
<div id="abstract">

**Abstract:** Discovered almost 10 years ago, the giant planet 51 Eridani b is one of the least separated ( $\approx$ 0.2 $\as$ ) and faintest (J $\approx$ 19.74 mag) directly imaged exoplanets known to date.  Its atmospheric properties have been thoroughly investigated through low- and medium-resolution spectroscopic observations, enabling robust characterization of the planet’s bulk parameters. However, the planet’s intrinsically high contrast renders high-resolution spectroscopic observations difficult, despite their potential to yield key measurements essential for a more comprehensive characterization. This study seeks to constrain the planet’s radial velocity, enabling a full 3D orbital solution when integrated with previous measurements. We have obtained 4 high-contrast high-resolution (R $\approx$ 140 000) spectroscopic datasets of the planet, collected over a two-year interval with the HiRISE visitor instrument at the VLT to derive the planet's radial velocity. Using self-consistent models of atmosphere, we were able to derive the radial velocity of the planet at each of the 4 epochs. These radial velocity measurements were then used in combination with all existing relative astrometry in order to constrain the orbit of the planet.Our radial velocity measurements allow us to break the degeneracy along the line of sight, making it now possible the unambiguous interpretation of the phase curve of the companion. We further constrain the orbital parameters, particularly the eccentricity, for which we derive e = $0.55_{-0.07}^{+0.03}$ . The relatively high eccentricity indicates that the system has experienced dynamical interactions induced by an external perturber. We place constraints on the mass and semi-major axis of a hypothetical, unseen outer planet capable of producing the observed high eccentricities.

</div>

<div id="div_fig1">

<img src="tmp_2602.10260/./Figures/astrometry_RV.png" alt="Fig4" width="100%"/>

**Figure 4. -** Orbit of 51 Eri b. The left panel shows the predicted RV of the planet as a function of time. The HiRISE RV measurements are overplotted as red dots, along with the rejected orbits in grey from the addition of the RV measurements. The middle panel is a zoomed-in version of the left panel around our RV measurements. In particular, we see that a future RV measurement at the end of 2026 should enable us to reject the last low-eccentric orbital populations. This is illustrated by the dashed vertical line overplotted at the end of 2026. The right panel displays the astrometric orbit of the planet inferred, including the line of nodes and the phase of the planet. The red crosses represent the estimate of the position of the planet during the $\hirise$ observations. (*fig:astrometry*)

</div>
<div id="div_fig2">

<img src="tmp_2602.10260/./Figures/corner_51Eri.png" alt="Fig6" width="100%"/>

**Figure 6. -** Posterior distribution of the inferred parameters of 51 Eri b. The posterior distribution in red was obtained including all available relative astrometry measurements. The posterior distribution in blue includes the HiRISE RV measurements. (*fig:corner*)

</div>
<div id="div_fig3">

<img src="tmp_2602.10260/./Figures/Fringes.png" alt="Fig2" width="100%"/>

**Figure 2. -** Power-spectral density (PSD) of the data filtered from the stellar speckles estimate of our four epochs. The PSD was lowpass filtered in order to focus on its low-frequency content. A secondary axis is displayed at the top, which corresponds to an equivalent velocity. A peak is visible around a resolution of 25 000 for each epoch, which corresponds to $\sim$12 $\kms$. This peak is particularly prominent in the data from the last epoch, when the planet was closer to the host star. (*fig:fringes*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.10260"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vect}[1]{\mathbf{#1}}$
$\newcommand{\rot}[1]{\vect{\nabla}\times{#1}}$
$\newcommand$
$\newcommand{\sref}[1]{Sec.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\equ}[1]{Eq.~(\ref{#1})}$
$\newcommand{\equo}[1]{Eq.~\ref{#1}}$
$\newcommand{\equs}[2]{Eqs.~(\ref{#1})~-~(\ref{#2})}$
$\newcommand{\equos}[2]{Eqs.~\ref{#1}~-~\ref{#2}}$
$\newcommand{\Msunpyr}{\mathrm{M_\odot/yr}}$
$\newcommand{\colout}[1]{\bgroup\markoverwith{\textcolor{#1}{\rule[.5ex]{2pt}{0.4pt}}}\ULon}$
$\newcommand{\pder}[2][]{\frac{\partial#1}{\partial#2}}$
$\newcommand{\grad}[1]{\nabla{#1}}$
$\newcommand{\div}[1]{\nabla\cdot\mathchoice{#1}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\cs}{c_{\sf s}}$
$\newcommand{\Teff}{T_{\sf eff}}$</div>



<div id="title">

# The role of detailed gas and dust opacities in shaping the evolution of the inner disc edge subject to episodic accretion 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.10674-b31b1b.svg)](https://arxiv.org/abs/2602.10674)<mark>Appeared on: 2026-02-12</mark> -  _22 pages, 15 figures, accepted for publication in A&A_

</div>
<div id="authors">

<mark>M. Cecil</mark>, et al. -- incl., <mark>M. Flock</mark>, <mark>P. Sudarshan</mark>

</div>
<div id="abstract">

**Abstract:** The transition in turbulence in the inner regions of protoplanetary discs and the closely connected dust sublimation front lead to periodic instability, manifesting as episodic accretion outbursts. For the corresponding interplay between heating and cooling, the opacity of the material needs to be treated carefully. We investigate the effects of different dust and gas opacity descriptions on the structure and evolution of the inner regions of protoplanetary discs. The influence on the episodic instability of the inner disc edge is hereby of central interest. Two-dimensional (2D) axisymmetric radiation hydrodynamic models are employed to simulate the evolution of the inner disc over the course of several thousand years. Our simulations greatly expand on previously published models by implementing detailed descriptions of the gas and dust opacities in terms of both their mean and frequency-dependent values. This allows us to also consider binned frequency-dependent irradiation from the central star. The adaptive opacity description impacts the structure of the inner disc rim to a great extent, with the gas opacities' contribution having the most significant influence. The resulting effects include the shift in position of both the dust sublimation front and the dead zone inner edge (DZIE), a significantly altered temperature in the dust-free region and the manifestation of the equilibrium temperature degeneracy as a sharp temperature transition. The episodic instability due to the activation of the magneto-rotational instability (MRI) in the dead zone still occurs, but at lower inner disc densities. While the gas opacities set the initial conditions for the instability by determining the location of the DZIE, the evolution of the outburst itself is mainly governed by the dust opacities. The analysis of criteria for non-axisymmetric instabilities reveals possible breaking of the density peaks produced during the burst phase. However, due to the periodicity of the instability cycle, the DZIE itself may remain stable throughout the quiescent phases according to linear criteria applied to our axisymmetric models. Although the thermal structure of the inner disc is crucially affected by different opacity descriptions, especially by the contribution of gas, the mechanism of the periodic instability of the DZIE remains active and is only marginally influenced by the gas opacities. The observational consequences of the severely altered temperatures may be significant and require further investigation.

</div>

<div id="div_fig1">

<img src="tmp_2602.10674/./figures/fig_burststructure_h2rmid_newopac.png" alt="Fig12" width="100%"/>

**Figure 12. -** Evolution of the outburst in the \texttt{FULL} model compared to $\texttt{MREF}^*$. The different columns correspond to different evolutionary stages, starting from the ignition of the burst at $t=t_\mathrm{TI}$, chronologically proceeding through the burst stage and ending with the beginning of the next quiescent state at $t=t_\mathrm{quies}$. The top row shows the temperature maps of \texttt{FULL} with the black and green contour lines marking the MRI-transition and the dust sublimation front. The middle row depicts the surface densities for both $\texttt{MREF}^*$ and \texttt{FULL} at the same stages during their respective evolution. The vertical dotted lines in the first panel indicate the locations of the DZIE in the corresponding model of the same colour. The panels in the second, third and fourth column also show the respective profiles of $\Sigma_\mathrm{min}^\mathrm{crit}$ together with a reference power law profile of $r^{0.7}$. The bottom row represents the aspect ratios $H/r$ of both models. The $r^{1/4}$ profile indicates the slope of $H/r$ in the outer disc (>2 AU). In the high-state regions, an additional reference profile of $r^{0.48}$ is shown. (*fig:burststructure*)

</div>
<div id="div_fig2">

<img src="tmp_2602.10674/./figures/fig_old_new_diff.png" alt="Fig3" width="100%"/>

**Figure 3. -** Differences between the models \texttt{DUST} and \texttt{FULL} during quiescence (panel a) and outburst (panel b). The coloured contour lines represent the same transitions as in Fig. \ref{fig:MREF_DUST_diff}, with the solid lines corresponding to \texttt{DUST} and the dashed lines to \texttt{FULL}. The upper halves of the panels show the differences in temperature between the two models, while the lower halves depict the absolute temperature of \texttt{DUST}.  (*fig:old_new_diff*)

</div>
<div id="div_fig3">

<img src="tmp_2602.10674/./figures/fig_rootfinder_combined_new.png" alt="Fig14" width="100%"/>

**Figure 14. -** Visualisation of the temperature jump as a consequence of the equilibrium temperature degeneracy. Panels (a) and (c) display temperature maps of the quiescent phase for the models \texttt{FULL} and \texttt{NOFREQIRR}, respectively. The red dashed lines show the radial profiles of the temperatures at a height of $z/r=0.2$(indicated by the white dotted line). The solution of Eq. \ref{eq:T_eq} for $T_\mathrm{eq}$ at every radius is represented by the blue line. The green contours mark the dust sublimation fronts. Panels (b) and (d) show the function given in Eq. \ref{eq:f_Teq}, evaluated at four different radii for both models, respectively. The star-shaped markers indicate which of the available solutions is adopted by the solver at each radius. The same markers have been used to locate the respective evaluation radii in panels (a) and (c). (*fig:rootfinder*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.10674"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\change}[1]{\textbf{#1}}$
$\newcommand{\teff}{\rm T_{eff}}$
$\newcommand{\logg}{\log(g)}$
$\newcommand{\feh}{\rm[Fe/H]}$
$\newcommand{\mh}{\rm[M/H]}$
$\newcommand{\micro}{\xi_{\rm micro}}$
$\newcommand{\macro}{\xi_{\rm macro}}$
$\newcommand{\micron}{\mu}$
$\newcommand{\loggf}{\log(gf)}$
$\newcommand{\exc}{E_{\rm low}}$
$\newcommand{\apoteff}{\textsc{TEFF}}$
$\newcommand{\apologg}{\textsc{LOGG}}$
$\newcommand{\apofeh}{\textsc{FE\_H}}$
$\newcommand{\apomh}{\textsc{M\_H}}$
$\newcommand{\apomhuncal}{\textsc{M\_H}_{\rm unc.}}$
$\newcommand{\apoalpha}{\textsc{ALPHA\_M}}$
$\newcommand{\apoparam}{\textsc{PARAM}}$
$\newcommand{\apofparam}{\textsc{FPARAM}}$
$\newcommand{\starbad}{\textsc{STAR\_BAD}}$
$\newcommand{\chibad}{\textsc{CHI2\_BAD}}$
$\newcommand{\metalsbad}{\textsc{METALS\_BAD}}$
$\newcommand{\metalswarn}{\textsc{METALS\_WARN}}$
$\newcommand{\RCG}{\rm R_{GC}}$</div>



<div id="title">

# The metal-poor tail of the APOGEE survey: I. Uncovering $\feh < -2.5$  stars from the inner Galaxy to the Magellanic Clouds 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.10859-b31b1b.svg)](https://arxiv.org/abs/2602.10859)<mark>Appeared on: 2026-02-12</mark> - 

</div>
<div id="authors">

M. Montelius, et al. -- incl., <mark>N. Martin</mark>

</div>
<div id="abstract">

**Abstract:** The most metal-poor stars in our Galaxy contain important clues of its earliest history, particularly those occupying the inner regions of the Galaxy. In the search for such metal-poor stars, large spectroscopic surveys are an invaluable tool. However, the spectra of metal-poor stars can be difficult to analyse because of the relative lack of available lines, which can also lead to misclassification. We aim to identify the stars observed by the APOGEE survey that are below the metallicity limit of APOGEE's analysis. For the highest confidence candidates, we classify the orbital properties of the stars to investigate whether their orbital distribution matches what we would expect for stars that are this metal poor and to select especially interesting targets for spectroscopic follow-up purposes. We examined the properties derived by APOGEE for metal-poor stars from the literature to find signatures of stars with a metallicity below the range of the grid used for spectral analysis. Once identified within APOGEE, we computed the orbits of our metal-poor candidates using AGAMA. The calibrated APOGEE stellar parameters provide a clear signature of the most metal-poor stars in the survey, indicated by null values for their metallicities while having effective temperatures and surface gravities determined by the pipeline. From comparison with the literature, we find that, within a temperature range of 3700 - 6700 K and above a threshold of S/N > 30, the vast majority of APOGEE stars without calibrated metallicities are very metal poor. Additional cleaning by visual inspection of the spectra improved the purity of the sample further. The radial velocities provided by APOGEE, Gaia DR3 positions and astrometry along with spectrophotometric distances derived in this work allowed for the computation of their orbits. In this work, we carefully selected 289 very metal-poor red giant stars (likely below $\feh$ = -2.5) from the APOGEE results. Our sample contains 16 very metal-poor member candidates of the Magellanic Clouds, 14 very metal-poor stars with orbits confined to the inner Galaxy, and 13 inner Galaxy halo interlopers. These samples significantly add to the very metal-poor stars known in these regions and allow for a more detailed picture of early chemical evolution across different environments.

</div>

<div id="div_fig1">

<img src="tmp_2602.10859/./figures/APOGEE_AV_distance_plxe_5.png" alt="Fig13" width="100%"/>

**Figure 13. -** 
    Left panel: Kiel diagram showing a BASTI isochrone corresponding to $\mh = -2.5$ and an age of 10 Gyr in black. Stars from the high confidence sample with $\frac{\pi}{\Delta\pi}>5$ to which distances have been determined are shown colour-coded by their offset from the distance determined by parallax. Right panel: Plot of the difference between our distances and the parallax distances plotted against parallax distance for the same stars as the left panel.  (*fig:AV_dist_comp*)

</div>
<div id="div_fig2">

<img src="tmp_2602.10859/./figures/dyn_groups_medApo_Zmax_E_Lz_dgfix_cloud_MC_Lz_fix.png" alt="Fig7" width="100%"/>

**Figure 7. -** Above: Dynamical groupings following [Viswanathan, et. al (2025)](https://ui.adsabs.harvard.edu/abs/2025A&A...695A.112V). We note that we used the median apocentres, which (due to the propagated uncertainties in the orbit integration) causes a bit of scatter around the diagonal line.
    Below: Energy versus the z component of the angular momentum. Likely members of a dwarf galaxies based on classifications from [Battaglia, et. al (2022)](https://ui.adsabs.harvard.edu/abs/2022A&A...657A..54B) have been marked with crossed out circles (red for Magellanic Cloud members, black for other satellite galaxies) as their orbits in a Milky Way potential cannot be trusted.  (*fig:dyn_group*)

</div>
<div id="div_fig3">

<img src="tmp_2602.10859/./figures/APOGEE_SH_distance_plxe_5.png" alt="Fig14" width="100%"/>

**Figure 14. -** Same comparisons as in Figure \ref{fig:AV_dist_comp} but for the _StarHorse_ distances instead of our work. The reduction in sample size is due to stars lacking _StarHorse_ distances. (*fig:SH_dist_comp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.10859"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\change}[1]{\textbf{#1}}$
$\newcommand{\teff}{\rm T_{eff}}$
$\newcommand{\logg}{\log(g)}$
$\newcommand{\feh}{\rm[Fe/H]}$
$\newcommand{\mh}{\rm[M/H]}$
$\newcommand{\micro}{\xi_{\rm micro}}$
$\newcommand{\macro}{\xi_{\rm macro}}$
$\newcommand{\micron}{\mu}$
$\newcommand{\loggf}{\log(gf)}$
$\newcommand{\exc}{E_{\rm low}}$
$\newcommand{\apoteff}{\textsc{TEFF}}$
$\newcommand{\apologg}{\textsc{LOGG}}$
$\newcommand{\apofeh}{\textsc{FE\_H}}$
$\newcommand{\apomh}{\textsc{M\_H}}$
$\newcommand{\apomhuncal}{\textsc{M\_H}_{unc.}}$
$\newcommand{\apoalpha}{\textsc{ALPHA\_M}}$
$\newcommand{\apoparam}{\textsc{PARAM}}$
$\newcommand{\apofparam}{\textsc{FPARAM}}$
$\newcommand{\starbad}{\textsc{STAR\_BAD}}$
$\newcommand{\chibad}{\textsc{CHI2\_BAD}}$
$\newcommand{\metalsbad}{\textsc{METALS\_BAD}}$
$\newcommand{\metalswarn}{\textsc{METALS\_WARN}}$</div>



<div id="title">

# The metal-poor tail of the APOGEE survey: II. Spectral analysis of Mg and Si in very metal-poor APOGEE spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.10883-b31b1b.svg)](https://arxiv.org/abs/2602.10883)<mark>Appeared on: 2026-02-12</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

M. Montelius, et al. -- incl., <mark>N. Martin</mark>

</div>
<div id="abstract">

**Abstract:** Very metal-poor stars are important tracers of the early chemical evolution history of the Milky Way. Infrared H-band spectroscopic surveys, such as APOGEE, are less affected by extinction in the more dust-obscured environments of our Galaxy. However, H-band spectra contain very limited spectral information for stars at the most metal-poor tail ( [ Fe/H ] < -2.5) because the available Fe lines in FGK stars in this wavelength range are weak. The first paper in this series successfully identified a sample of 327 very metal-poor stars (with [ Fe/H ] $< -2$ ) from the APOGEE database, 289 of which are on the red giant branch. The spectra of these stars were not properly analysed by the APOGEE main pipeline because they are very metal poor. In this work, we measure metallicities for these stars using the abundances of the elements Mg and Si. We demonstrate that the absorption lines of the elements Mg and Si are of good quality despite the challenging combination of (low) metallicity, wavelength regime, spectral resolution, and signal-to-noise ratios available for these spectra. A specialised pipeline was designed to measure the abundance of Mg and Si in APOGEE spectra and  yielded a robust estimate of the overall metallicity. In order to provide reliable measurements, we tested three different sets of assumptions for Mg and Si enhancement. We present Mg and Si abundances as well as overall metallicities for 327 stars, all of which had previously gotten null values from the main APOGEE pipeline for either the calibrated [ M/H ] or [ Fe/H ] . The typical uncertainties for our measurements are 0.2 dex. We found five stars in our sample with unusual [ Si/Mg ] abundances higher than 0.5, and we connect this signature to globular cluster stars, and this might be related to specific supernova events. Our data suggest a concentration of high [ Si/Mg ] stars in the Sextans dwarf galaxy. Other dwarf galaxies are found to agree well with results in the literature. Our derived metallicities range between $-3.1\leq[M/H]\leq-2.25$ , thereby pushing the metal-poor tail of APOGEE results down by 0.6 dex.

</div>

<div id="div_fig1">

<img src="tmp_2602.10883/./xmatch_both.png" alt="Fig4" width="100%"/>

**Figure 4. -** [Si/Mg] vs. [Mg/H] for the GC stars in our sample (shown in black), along with APOGEE abundances for the same GCs (shown with the same symbols as our measurements). The dotted lines indicate y=x lines for each cluster that match the cluster stars at high [Si/Mg] values. (*fig:GlobCluster*)

</div>
<div id="div_fig2">

<img src="tmp_2602.10883/./SAGA_Si_Mg_hires_errorbar_nogrid.png" alt="Fig5" width="100%"/>

**Figure 5. -** [Si/Mg] for stars from our sample in various dwarf galaxies (as identified by the criteria of \cite{Battaglia2022DG}; black symbols) supplemented with SAGA dwarf galaxy members according to the same criteria. Only measurements from high-resolution spectra are shown. Chemical evolution models for the Solar neighbourhood and a UFD are shown in all panels, with dedicated models for Carina, Fornax, Sculptor and Sextans shown in their respective panels. (*fig:SAGA_Si_Mg*)

</div>
<div id="div_fig3">

<img src="tmp_2602.10883/./APO_main_high_hist_combined.png" alt="Fig2" width="100%"/>

**Figure 2. -** Residual plot for estimated metallicities against photometric and spectroscopic metallicities, shown as a histogram. The means are shown as dashed lines, and given along with the standard deviations.  (*fig:comparisons*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.10883"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

123  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
